### Песочница
    https://habr.com/ru/post/418281/

In [2]:
import sqlite3
from bs4 import BeautifulSoup

In [3]:
# обработка базы данных

original_db = 'C:\\Users\\busya\\Desktop\\anaconda\\проекты\\курсовая\\sberbank_hh.db'
new_db = 'C:\\Users\\busya\\Desktop\\anaconda\\проекты\\курсовая\\updated_vacancies.db'


conn = sqlite3.connect(new_db)
cursor = conn.cursor()

#cursor.execute('SELECT * from vacancy')
cursor.execute('SELECT * from vacancies')
results = cursor.fetchall()

conn.close()

In [4]:
len(results)

1816

In [8]:
results[0][1]

'Менеджер по продажам продуктов эквайринга'

### Создание новой базы данных
    > удаление дубликатов
    > приведение к более удобочитаемому виду

In [6]:
conn = sqlite3.connect('updated_vacancies.db')
cursor = conn.cursor()

In [181]:
def get_descr(raw_html):
    soup = BeautifulSoup(raw_html)
    return soup.find('div', class_='g-user-content').text.strip()

In [154]:
conn = sqlite3.connect('C:\\Users\\busya\\Desktop\\anaconda\\проекты\\курсовая\\updated_vacancies.db')
cursor = conn.cursor()
cursor.execute('''CREATE TABLE vacancies (link text, name text, city text, release_date text, description text)''')

In [155]:
links = []
errors = []

for vacancy in results:
    
    if vacancy[1] in links:
        continue
    
    else:
        links.append(vacancy[1])
        vac_set = (
                    vacancy[1], # link
                    vacancy[2], # vacancy name
                    vacancy[3].split('<p>')[3][8:-4], # city name
                    vacancy[4][:10], # date
                    get_descr(vacancy[5]) # description
        )
        try:
            cursor.execute(f'''INSERT INTO vacancies VALUES {vac_set}''')
        except:
            errors.append(vacancy[1])
conn.commit()

In [166]:
errors # некорректное описание (использованы спецсимволы)

['https://hh.ru/vacancy/27408489', 'https://hh.ru/vacancy/29810176']

In [167]:
cursor.execute('SELECT * from vacancies')
updated_res = cursor.fetchall()
conn.close()

In [168]:
len(results) - len(updated_res) # удалено вакансий

444

### Что можно сделать с помощью nltk, word2vec?

In [216]:
import nltk
import string
from nltk.corpus import stopwords
import re

In [186]:
test_descr = [vacancy[4] for vacancy in results[:10]]

In [239]:
text = test_descr[2]

In [240]:
text.split('   ')

['Если ты:  хочешь построить карьеру в банковском секторе; стремишься стать лидером и развиваться профессионально и личностно; нацелен на достижение лучших результатов для клиентов; мечтаешь работать в инновационной команде профессионалов.',
 ' Начни карьеру в крупнейшем банке России и СНГ!',
 'Как консультант, ты будешь:  продавать онлайн-продукты и услуги банка клиентам в отделении, используя планшет и терминалы самообслуживания (Автоплатеж, Онлайн-вклады, Мобильный банк); объяснять клиентам преимущества использования устройств самообслуживания и проводить обучения по работе с ними; выявлять цели визита клиента в банк и направлять его к соответствующим специалистам отделения.',
 ' Ты подходишь нам, если ты:  специалист со средним профессиональным/неполным высшим образованием; владеешь программами Microsoft Office, Internet Explorer, Outlook; выдерживаешь высокий темп работы; готов работать в операционном зале в течение всего рабочего дня; доброжелательно относишься к окружающим, гото

In [256]:
results[1][0]

'https://hh.ru/vacancy/29655166'

In [253]:
results[2][0]

'https://hh.ru/vacancy/29655039'

In [283]:
results[1245][4].split('.')

['Обязанности:  увеличение объемов бизнеса с закрепленными клиентами CIB (крупнейшего бизнеса) и обеспечение их комплексного обслуживания с учетом индивидуальных потребностей, вне зависимости от места их регистрации и/или территории совершения операции; обеспечение максимальной удовлетворенности клиентов CIB за счет первоклассного качества обслуживания и предоставления полностью индивидуализированных решений на базе широкой продуктовой линейки; организация работы команды с целью реализации бизнес-задач/проектов',
 '    Требования:  высшее экономическое/финансовое образование; опыт работы в корпоративном бизнесе не менее 3-х лет; обязателен опыт работы в области кредитования юридических лиц и/или в области продаж корпоративным клиентам от 2-х лет; знание и понимание разных отраслей экономики; знание английского языка не ниже уровня Intermediate; навыки управления проектами/командой; нацеленность на результат; высокий уровень коммуникативных навыков',
 '  Условия работы:  работа в крупне

In [532]:
def check_it(text):
    ans = False
    possible_definitions = ['Ты подходишь нам, если ты', 'Требования', 'Навыки', 'Технические компетенции', 'Если есть',
                           'ТРЕБОВАНИЯ К КАНДИДАТУ', 'Необходимые навыки и технологии', 'Что мы ждем от кандидата',
                           'Мы ожидаем от Вас', 'Мы ждем, что ты', 'Профессиональные компетенции', 'Ты подходишь нам, если',
                           'Профессиональные навыки', 'Мы ждём от тебя', 'Мы ожидаем, что ты', 'Наши ожидания',
                           'Что мы хотим в вас видеть?', 'От Вас нужно', 'Стэк',
                           'Мы определили следующие обязательные требования к кандидату', 'Требование', 'Общие требования',
                           'Требуемые навыки', 'Профессиональные требования', 'Обязательно', 'От вас нужно', 'Чего ждем от кандидата',
                           'Что ждем', 'Опыт работы с технологиями', 'Мы ожидаем', 'ТЕХНИЧЕСКИЕ КОМПЕТЕНЦИИ', 'ТРЕБОВАНИЯ',
                           'Знание технологий', 'Опыт:', 'Обязательные требования', 'Мы ждем что вы умеете/обладаете']
    
    for def_ in possible_definitions:
        if def_ in text:
            ans = True
            break
    return ans

In [533]:
simple_texts = 0
other = []
for i in range(len(results)):
    if check_it(results[i][4]):
        simple_texts += 1
    else:
        other.append(i)

In [534]:
simple_texts

1772

In [535]:
other[:10]

[99, 268, 273, 301, 391, 443, 578, 670, 745, 746]

In [536]:
results[711][4]

'Кто же мы?Мы – команда разработки Apache Ignite, распределенной базы данных с открытым исходным кодом, которую, в своих продуктах используют тысячи разработчиков во всем мире. Она так же является сердцем многих наших сервисов, которыми уже завтра будут пользоваться миллионы клиентов по всей России.Больше про Apache Ignite: ignite.apache.orgЧем придется заниматься?    Нагрузочным тестированием;   Подготовкой профилей нагрузки;   Анализом результатов тестов и поиском дефектов, вызывающих деградацию производительности.   Изучением нового, ведь часто задачи, с которыми вам придется столкнутся, не имеют типовых решений.   Мы ждем что вы умеете/обладаете:   Опытом тестирования от трех лет;   Опытом нагрузочного тестирование от одного года;   Опытом подготовки профилей нагрузки;   Знанием теории и методик тестирования;   Опытом в автоматизации (Java/Python);   Знанием основ Linux/Unix.   Круто если вы так же:  Имеет опыт работы с Apache Ignite; Имеете опыт работы/тестирования распределенных 

In [ ]:
nontrivial = [194, 220]

### Что нужно научиться делать?
    > отделять технические профессии от нетехнических
    > выделять заголовки подтем

In [11]:
job_names = {} # словарь профессий с записью индексов повторяющихся позиций
for i in range(len(results)):
    name = results[i][1]
    if name in job_names:
        job_names[name].append(i)
    else:
        job_names[name] = [i]
        

In [23]:
list(job_names.keys())[:10]

['Менеджер по продажам продуктов эквайринга',
 'Консультант',
 'Инженер технической поддержки автоматизированных рабочих мест',
 'Frontend разработчик',
 'JavaScript разработчик',
 'Ведущий инженер',
 'Специалист по прямым продажам',
 'Специалист по кадрам',
 'Менеджер по продажам',
 'Директор филиала лизинговой компании']

In [21]:
job_lengthes = {}
for job in list(job_names.keys()):
    job_lengthes[job] = len(job_names[job])

In [28]:
job_lengthes

{'Менеджер по продажам продуктов эквайринга': 14,
 'Консультант': 300,
 'Инженер технической поддержки автоматизированных рабочих мест': 1,
 'Frontend разработчик': 1,
 'JavaScript разработчик': 1,
 'Ведущий инженер': 4,
 'Специалист по прямым продажам': 94,
 'Специалист по кадрам': 4,
 'Менеджер по продажам': 138,
 'Директор филиала лизинговой компании': 1,
 'Специалист по взысканию задолженности': 26,
 'Клиентский менеджер (ипотечное направление)': 1,
 'Старший менеджер по обслуживанию': 57,
 'Клиентский менеджер малого бизнеса': 9,
 'Начальник сектора дистанционных продаж и ипотечного кредитования': 1,
 'Руководитель проекта': 1,
 'Старший менеджер по обслуживанию (п. Ильинский)': 1,
 'Главный специалист сектора сопровождения стандартных сделок': 1,
 'Главный инженер по сопровождению': 3,
 'Аналитик': 6,
 'Специалист отдела оформления стандартных сделок': 1,
 'Специалист по сопровождению зарплатных проектов': 2,
 'Full stack developer': 2,
 'Аналитик HCM (PA, OM)': 1,
 'Дизайнер UX|

In [31]:
sorted_ = sorted(job_lengthes, key = lambda x: job_lengthes[x], reverse=True)
sorted_

['Консультант',
 'Менеджер по продажам',
 'Специалист по прямым продажам',
 'Старший менеджер по обслуживанию',
 'Менеджер по работе с ключевыми клиентами',
 'Менеджер по обслуживанию',
 'Специалист по взысканию задолженности',
 'Клиентский менеджер Сбербанк Премьер',
 'Java-разработчик',
 'Менеджер по продажам продуктов эквайринга',
 'Клиентский менеджер',
 'Старший специалист по взысканию',
 'Системный аналитик',
 'Java разработчик',
 'Клиентский менеджер малого бизнеса',
 'Старший кредитный аналитик',
 'Менеджер по работе с ключевыми клиентами малого бизнеса',
 'Водитель-инкассатор',
 'Кредитный инспектор',
 'Кассир',
 'Юрисконсульт',
 'Главный инженер по разработке',
 'Старший юрисконсульт',
 'Клиентский менеджер отдела продаж лизинга крупному и среднему бизнесу',
 'Водитель',
 'Аналитик',
 'Старший специалист',
 'Сервис-менеджер',
 'Менеджер по ипотечному кредитованию',
 'Старший специалист по взысканию задолженности',
 'Ведущий инженер по разработке',
 'Руководитель направления',